# Basic Tensorflow 2.0 Coding Style

In [1]:
import tensorflow as tf
tf.executing_eagerly()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus: tf.config.experimental.set_memory_growth(gpus[0], True)

## Make DataSet

In [3]:
'''
If you have numpy data, you can use
tensor_data = tf.convert_to_tensor(numpy_data, dtype=tf.float32)
for translation into tf.Tensor.
'''
# example training data
feature = tf.random.normal(shape=[50000, 1000])
target = tf.random.normal(shape=[50000, 10])

# example validation data
val_feature = tf.random.normal(shape=[10000, 1000])
val_target = tf.random.normal(shape=[10000, 10])

# example test data
test_feature = tf.random.normal(shape=[5000, 1000])
test_target = tf.random.normal(shape=[5000, 10])


# make dataset
dataset = tf.data.Dataset.from_tensor_slices((feature, target))
val_dataset = tf.data.Dataset.from_tensor_slices((val_feature, val_target))
test_dataset = tf.data.Dataset.from_tensor_slices((test_feature, test_target))

In [4]:
# A dataset have shape information except batchsize and data type.
dataset

<TensorSliceDataset shapes: ((1000,), (10,)), types: (tf.float32, tf.float32)>

In [5]:
# Training data should be shuffled every epoch.
# 10000 is buffer size.
dataset = dataset.shuffle(10000)

# For mini-batch training.
# 256 is batch size.
dataset = dataset.batch(256)

# Of course we can write same code as follows
# dataset = dataset.shuffle(10000).batch(256)

# validation data and test data do NOT need shuffle.
# batch size is as big as possible.
val_dataset = val_dataset.batch(10000)
test_dataset = test_dataset.batch(5000)

In [6]:
# dataset is set for batch training.
dataset

<BatchDataset shapes: ((None, 1000), (None, 10)), types: (tf.float32, tf.float32)>

In [7]:
class MyNet(tf.keras.Model):
    '''
    We use basically tf.keras.Model for making network.
    This class will manage layers and that's trainable parameters.
    '''
    def __init__(self):
        super(MyNet, self).__init__()
        
        
        # We can use tf.keras.Sequential 
        # which has a role of putting together some layers.
        # This class inherits tf.keras.Model, so this can manege parameters too.
        # This class only receive layers.Layer class.
        # (Note that tf.keras.Sequential receive tf.keras.layers.ReLU())
       
        self.layer1 = tf.keras.Sequential([
            tf.keras.layers.Dense(1024),
            tf.keras.layers.ReLU(),
            tf.keras.layers.BatchNormalization(axis=-1),
            tf.keras.layers.Dropout(rate=0.2),
        ])
        
        # Of course we can write some layers separately.
        
        self.dense = tf.keras.layers.Dense(256)
        self.bn = tf.keras.layers.BatchNormalization(axis=-1)
        self.do = tf.keras.layers.Dropout(rate=0.2)
        
        self.dense_output = tf.keras.layers.Dense(10)
    
    # tf.function is jit compiler which translate python code into TF graph.
    @tf.function
    def call(self, x, training=False):
        # tf.keras.Sequential class have training propaty
        # which manege behavior of dropout and batchnormalization etc.
        h = self.layer1(x, training=training)
        
        h = self.dense(h)
        # we can use tf.nn.relu function instead of tf.keras.layers.ReLU()
        h = tf.nn.relu(h)

        # BatchNormalization and Dropout class also have training property.
        h = self.bn(h, training=training)
        h = self.do(h, training=training)
        
        return self.dense_output(h)

In [11]:
model = MyNet()
# test execution.
model(tf.random.normal(shape=[1, 1000]))

model.summary()

Model: "my_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    multiple                  1029120   
_________________________________________________________________
dense_7 (Dense)              multiple                  262400    
_________________________________________________________________
batch_normalization_5 (Batch multiple                  1024      
_________________________________________________________________
dropout_5 (Dropout)          multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  2570      
Total params: 1,295,114
Trainable params: 1,292,554
Non-trainable params: 2,560
_________________________________________________________________


## Training by hand

In [17]:
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.MeanSquaredError()

In [18]:
@tf.function
def train_step(feature, target):

    with tf.GradientTape() as tape:
        y_pred = model(feature, training=True)
        loss = loss_fn(target, y_pred)
    
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    
    return loss

@tf.function
def val_step(feature, target):
    
    y_pred = model(feature)
    loss = loss_fn(target, y_pred)
    
    return loss

In [20]:
for i in range(10):
    
    running_loss = 0
    running_val_loss = 0
    
    for i, (batch_feature, batch_target) in enumerate(dataset):
        loss_ = train_step(batch_feature, batch_target)
        running_loss += loss_
        
    for j, (batch_feature, batch_target) in enumerate(val_dataset):
        loss_ = val_step(batch_feature, batch_target)
        running_val_loss += loss_
        
    print("----------epoch {}--------".format(i+1))
    print("loss: {},  val_loss: {}".format(running_loss/(i+1), 
                                           running_val_loss/(j+1)))

----------epoch 196--------
loss: 0.46282273530960083,  val_loss: 1.203696846961975
----------epoch 196--------
loss: 0.45190444588661194,  val_loss: 1.2077049016952515
----------epoch 196--------
loss: 0.44128596782684326,  val_loss: 1.2036551237106323
----------epoch 196--------
loss: 0.43410274386405945,  val_loss: 1.2066048383712769
----------epoch 196--------
loss: 0.4279598295688629,  val_loss: 1.2057843208312988
----------epoch 196--------
loss: 0.41745299100875854,  val_loss: 1.2026267051696777
----------epoch 196--------
loss: 0.4114142954349518,  val_loss: 1.2086076736450195
----------epoch 196--------
loss: 0.40470507740974426,  val_loss: 1.2065753936767578
----------epoch 196--------
loss: 0.39911866188049316,  val_loss: 1.2081482410430908
----------epoch 196--------
loss: 0.3923533260822296,  val_loss: 1.207283616065979
